In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation,TruncatedSVD
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.



def createBasicFeatures(corpus):
  texts=[]
  classes=[]
  for i in range(len(corpus)):
    classes.append(corpus[i]['class'])
    texts.append(re.sub("[^a-zA-Z0-9]+", ' ',corpus[i]['text']))
  
  
  
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(texts)
  X=X.toarray()

  
  return X,classes,list(vectorizer.vocabulary_.keys())

def Create_stop(texts):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(texts)
  X=X.toarray()
  df=pd.DataFrame(X.sum(axis=0),columns=['Count'])
  df.sort_values(by='Count',ascending=False,inplace=True)

  most_common=list(df[:100].index)
  rr=list(vectorizer.vocabulary_.items())
  dict1={}
  for a,b in rr:
    dict1[b]=a


  common_words=[]
  for j in most_common:
        common_words.append(dict1[j])

  off=list(df[df['Count']==1].index)


  rare=[]
  for i in off:
    rare.append(dict1[i])

  stop=stopwords.words("english")
  stop.extend(common_words)
  stop.extend(rare)

  return(stop)

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here


  
  """  
  vectorizer = TfidfVectorizer(stop_words=set(stopwords.words("english")),min_df=0.02,max_features=35000)
  X = vectorizer.fit_transform(texts)
  X=X.toarray()

  cv = CountVectorizer()
  bo = cv.fit_transform(texts)
  bo=bo.toarray()
  lda = LatentDirichletAllocation(n_components=10,random_state=0)
  l=lda.fit_transform(bo)
  X=np.hstack((X,l))
  #X=np.hstack((X,bo))"""
  texts=[]
  classes=[]
  for i in range(len(corpus)):
    classes.append(corpus[i]['class'])
    texts.append(re.sub("[^a-zA-Z0-9]+", ' ',corpus[i]['text']))
    """t=re.sub("[^a-zA-Z0-9]+", ' ',corpus[i]['text'])
    dummy=[]
    for word in t.split():
      dummy.append(lemma.lemmatize(word))
    
    texts.append(" ".join(dummy))"""
    
  stop=Create_stop(texts)
  #vectorizer = CountVectorizer(stop_words=stop,ngram_range=(1,3),max_features=40000)#,max_features=20000,ngram_range=(1,2))
  vectorizer = CountVectorizer(ngram_range=(1,2),min_df=0.002)
  X = vectorizer.fit_transform(texts)
  X=X.toarray()
  
  
  """vectorizer1 = TfidfVectorizer(stop_words=stop)
  tf = vectorizer1.fit_transform(texts)
  tf = tf.toarray()
  tsd=TruncatedSVD(5)
  t=tsd.fit_transform(tf)

  X=np.hstack((X,t))
  for i in range(5):
    vectorizer.vocabulary_.update({'svd'+str(i):len(vectorizer.vocabulary_)+i})"""

  print("The number of features ={}".format(len(list(vectorizer.vocabulary_.keys()))))
  return X,classes,list(vectorizer.vocabulary_.keys())

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [0]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [0]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.825500
The most informative terms for pos are: ['drainage', 'motivator', 'sadistically', 'velicorapters', 'ephron', 'greenaway', 'shadowy', 'decked', 'tunie', 'kamen', 'endorsements', 'ferrets', 'deepak', 'kahl', 'tobacco', 'preteen', 'landou', 'filmic', 'chloe', 'lyndon']
The most informative terms for neg are: ['frickin', 'hillhouse', 'punched', 'simm', 'magoo', 'leonardi', 'whilst', 'caerthan', 'sioux', 'edifice', 'gremlins', 'millisecond', 'jiggling', 'overlays', 'minorly', 'second', 'stylewise', 'knockout', 'nickelodeon', 'fooled']
----------L2 Norm-----------
The model's average accuracy is 0.834000
The most informative terms for pos are: ['tobacco', 'bryan', 'york', 'deepak', 'anabasis', 'subtlty', 'decked', 'greenaway', 'motivator', 'endorsements', 'mixon', 'reemerging', 'sadistically', 'beats', 'infect', 'norrington', 'summoned', 'comprehendable', 'manero', 'velicorapters']
The most informative terms for neg are: [

Run the following to train and evaluate two models using extended features:

In [0]:
X,y,vocab = createFancyFeatures(corpus) 
runEvaluation(X, y, vocab)


The number of features =58168
----------L1 Norm-----------
The model's average accuracy is 0.833500
The most informative terms for pos are: ['it didn', 'experience with', 'is replaced', 'million people', 'the comedic', 'their souls', 'it before', 'ever seeing', 'imprisoned', 'redeemed', 'the confines', 'briefcase', 'pain for', 'unmistakable', 'comes at', 'james jennifer', 'gripe', 'as father', 'love movies', 'at small']
The most informative terms for neg are: ['minute it', 'klutz', 'else will', 'dewitt bukater', 'were almost', 'more that', 'and full', 'graham', 'boy does', 'and before', 'panders', 'charming in', 'hadn been', 'baldwin but', 'but many', 'realization', 'only been', 'in while', 'show what', 'the combination']
----------L2 Norm-----------
The model's average accuracy is 0.850500
The most informative terms for pos are: ['james jennifer', 'unmistakable', 'ring true', 'eyes and', 'would definitely', 'the surprise', 'others with', 'the fantastic', 'rangers', 'debate and', 'love